# Imports

In [1]:
from botiverse import chat_gui
from botiverse.bots import DeepTODS
from botiverse.models import BERTConfig, TRIPPYConfig
import gdown
import os

Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=1f2iOTT-QiFbIc1naqGVZWX5wPVo7gUMS
To: /Users/essam/Documents/GitHub/Botiverse/botiverse/models/TRIPPY/vocab.txt
100%|██████████| 231k/231k [00:00<00:00, 558kB/s]


Done.


Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=1f2iOTT-QiFbIc1naqGVZWX5wPVo7gUMS
To: /Users/essam/Documents/GitHub/Botiverse/botiverse/models/TRIPPY/vocab.txt
100%|██████████| 231k/231k [00:01<00:00, 197kB/s]


Done.


Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=1f2iOTT-QiFbIc1naqGVZWX5wPVo7gUMS
To: /Users/essam/Documents/GitHub/Botiverse/botiverse/models/TRIPPY/vocab.txt
100%|██████████| 231k/231k [00:01<00:00, 137kB/s]

Done.


# Download necessary files to run the example

In [2]:
# Download slot list
f_id = '1tRprClzgNCtVks-SgZilBD7fdc5W3XxY'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'slot_list.json'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Slot list downloaded successfully.')
else:
    print('Slot list already exists. Skipping download.')

Slot list already exists. Skipping download.


In [3]:
# Download label maps
f_id = '15guVrztM12KMMDp6jgu53ivUU945iRNp'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'label_maps.json'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Label maps downloaded successfully.')
else:
    print('Label maps already exists. Skipping download.')

Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=15guVrztM12KMMDp6jgu53ivUU945iRNp
To: /Users/essam/Documents/GitHub/Botiverse/examples/bots/TaskBot/sim-R_demo/label_maps.json
100%|██████████| 7.00/7.00 [00:00<00:00, 7.94kB/s]

Label maps downloaded successfully.


In [4]:
# Download DST weights trained on sim-R
f_id = '1POjBULmqxBrebvINfl989bskAstV3Zld'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'model.pt'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Model downloaded successfully.')
else:
    print('Model already exists. Skipping download.')

Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=1POjBULmqxBrebvINfl989bskAstV3Zld
To: /Users/essam/Documents/GitHub/Botiverse/examples/bots/TaskBot/sim-R_demo/model.pt
100%|██████████| 439M/439M [04:29<00:00, 1.63MB/s] 


Model downloaded successfully.


# Const

In [5]:
CHATBOT_NAME = 'Tody'

DOMAINS = ["restaurant"]

SLOT_LIST_PATH = './slot_list.json'

LABEL_MAPS_PATH = './label_maps.json'

POLICY = 'Priority' # Priority or Random

START = [
    {
        'utterance': 'Hi I am Tody, I can help you reserve a restaurant?',
        'slots': [],
        'system_act': {}
    }
]

TEMPLATES = [
    {
        'utterance': 'what type of food do you want and in what area?',
        'slots': ['restaurant-location', 'restaurant-category'],
        'system_act': {}
    },
    {
        'utterance': 'what is your preferred price range and rating?',
        'slots': ['restaurant-price_range', 'restaurant-rating'],
        'system_act': {}
    },
    {
        'utterance': 'how many people will be in your party?',
        'slots': ['restaurant-num_people'],
        'system_act': {}
    },
    {
        'utterance': 'what time and date would you like to reserve a table for?',
        'slots': ['restaurant-time', 'restaurant-date'],
        'system_act': {}
    },
    {
        'utterance': 'May I suggest kfc restaurant?',
        'slots': ['restaurant-restaurant_name'],
        'system_act': {'restaurant-restaurant_name': ['kfc']}
    },
    {
        'utterance': 'ok done, here is your reservation number: 123456',
        'slots': [],
        'system_act': {}
    }
]

NON_REFERABLE_SLOTS = []

NON_REFERABLE_PAIRS = []

FROM_SCRATCH = True

MODEL_PATH = './model.pt'

TRIPPY_CONFIG = TRIPPYConfig(multiwoz=False)

BERT_CONFIG = BERTConfig()

Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=1f2iOTT-QiFbIc1naqGVZWX5wPVo7gUMS
To: /Users/essam/Documents/GitHub/Botiverse/botiverse/models/TRIPPY/vocab.txt
100%|██████████| 231k/231k [00:01<00:00, 205kB/s]

Done.


# Build model & Load weights

In [6]:
tods = DeepTODS(CHATBOT_NAME, DOMAINS, ONTOLOGY_PATH, LABEL_MAPS_PATH, POLICY, START, TEMPLATES, NON_REFERABLE_SLOTS, NON_REFERABLE_PAIRS, FROM_SCRATCH, BERT_CONFIG, TRIPPY_CONFIG)
tods.load_dst_model(MODEL_PATH)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tods = DeepTODS(CHATBOT_NAME, DOMAINS, ONTOLOGY_PATH, LABEL_MAPS_PATH, POLICY, START, TE     │
│   2 tods.load_dst_model(MODEL_PATH)                                                              │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ONTOLOGY_PATH' is not defined

# Conversation

In [7]:
def fun(utter):

    sys = tods.infer(utter)

    sys = sys if sys is not None else ''

    return sys + '\n' + ' state:' + str(tods.get_dialogue_state())

chat_gui = chat_gui(fun)
chat_gui.run()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │                                                                                           │
│    7 │   return sys + '\n' + ' state:' + str(tods.get_dialogue_state())                          │
│    8                                                                                             │
│ ❱  9 chat_gui = chat_gui(fun)                                                                    │
│   10 chat_gui.run()                                                                              │
│   11                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: __init__() missing 1 required positional argument: 'chat_func'

In [8]:
tods.reset()
print(tods.get_dialogue_state())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tods.reset()                                                                                 │
│   2 print(tods.get_dialogue_state())                                                             │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [9]:
response = tods.infer('')
print(tods.get_dialogue_state())
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = tods.infer('')                                                                    │
│   2 print(tods.get_dialogue_state())                                                             │
│   3 print(response)                                                                              │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [10]:
response = tods.infer('Hi, I want to book a table for lunch.')
print(tods.get_dialogue_state())
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = tods.infer('Hi, I want to book a table for lunch.')                               │
│   2 print(tods.get_dialogue_state())                                                             │
│   3 print(response)                                                                              │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [11]:
response = tods.infer('I would like to have egyptian food in cairo.')
print(tods.get_dialogue_state())
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = tods.infer('I would like to have egyptian food in cairo.')                        │
│   2 print(tods.get_dialogue_state())                                                             │
│   3 print(response)                                                                              │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [12]:
response = tods.infer('I want a cheap restaurant with good rating.')
print(tods.get_dialogue_state())
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = tods.infer('I want a cheap restaurant with good rating.')                         │
│   2 print(tods.get_dialogue_state())                                                             │
│   3 print(response)                                                                              │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [13]:
response = tods.infer('I have 4 people in my party.')
print(tods.get_dialogue_state())
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = tods.infer('I have 4 people in my party.')                                        │
│   2 print(tods.get_dialogue_state())                                                             │
│   3 print(response)                                                                              │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [14]:
response = tods.infer('4 am on the tuesday.')
print(tods.get_dialogue_state())
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = tods.infer('4 am on the tuesday.')                                                │
│   2 print(tods.get_dialogue_state())                                                             │
│   3 print(response)                                                                              │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [15]:
response = tods.infer('ok, I love kfc.')
print(tods.get_dialogue_state())
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = tods.infer('ok, I love kfc.')                                                     │
│   2 print(tods.get_dialogue_state())                                                             │
│   3 print(response)                                                                              │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tods' is not defined

In [16]:
curr_dir = os.path.dirname(os.path.abspath(__file__))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 curr_dir = os.path.dirname(os.path.abspath(__file__))                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name '__file__' is not defined